# Spark Prepare Exercises

## Part 1

    1. Read the case.csv file from the 311 call data into a Spark DataFrame.
    2. How old is the latest (in terms of days past SLA) currently open issue? How long has the oldest (in terms of days since opened) currently opened issue been open?
    3. How many Stray Animal cases are there?
    4. How many service requests that are assigned to the Field Operations department (dept_division) are not classified as "Officer Standby" request type (service_request_type)?
    5. Create a new DataFrame without any information related to dates or location.
    6. Read dept.csv into a Spark DataFrame. Inspect the dept_name column. Replace the missing values with "other".

In [1]:
import pyspark
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, expr

In [2]:
spark = SparkSession.builder.master("local").appName("read").\
    enableHiveSupport().\
    getOrCreate()

In [3]:
df = spark.read.csv('./case.csv', header=True)

In [4]:
type(df)

pyspark.sql.dataframe.DataFrame

In [5]:
df.head(5)

[Row(case_id='1014127332', case_opened_date='1/1/18 0:42', case_closed_date='1/1/18 12:29', SLA_due_date='9/26/20 0:42', case_late='NO', num_days_late='-998.5087616000001', case_closed='YES', dept_division='Field Operations', service_request_type='Stray Animal', SLA_days='999.0', case_status='Closed', source_id='svcCRMLS', request_address='2315  EL PASO ST, San Antonio, 78207', council_district='5'),
 Row(case_id='1014127333', case_opened_date='1/1/18 0:46', case_closed_date='1/3/18 8:11', SLA_due_date='1/5/18 8:30', case_late='NO', num_days_late='-2.0126041669999997', case_closed='YES', dept_division='Storm Water', service_request_type='Removal Of Obstruction', SLA_days='4.322222222', case_status='Closed', source_id='svcCRMSS', request_address='2215  GOLIAD RD, San Antonio, 78223', council_district='3'),
 Row(case_id='1014127334', case_opened_date='1/1/18 0:48', case_closed_date='1/2/18 7:57', SLA_due_date='1/5/18 8:30', case_late='NO', num_days_late='-3.022337963', case_closed='YES',

In [6]:
df.orderBy(df.case_opened_date.desc()).where(df.case_closed == 'NO').show()

+----------+----------------+----------------+--------------+---------+-------------------+-----------+--------------------+--------------------+------------------+-----------+------------+--------------------+----------------+
|   case_id|case_opened_date|case_closed_date|  SLA_due_date|case_late|      num_days_late|case_closed|       dept_division|service_request_type|          SLA_days|case_status|   source_id|     request_address|council_district|
+----------+----------------+----------------+--------------+---------+-------------------+-----------+--------------------+--------------------+------------------+-----------+------------+--------------------+----------------+
|1013863552|    9/9/17 11:43|            null| 11/14/17 8:30|      YES|        47.64583333|         NO|    Code Enforcement|    Permits Building|       65.86542824|       Open|CRM_Listener|905  PRADO ST, Sa...|               5|
|1013860368|     9/8/17 9:47|            null|  1/26/18 9:47|       NO|       -25.408229

How many Stray Animal cases are there?

In [7]:
df.select('service_request_type').where(df.service_request_type == 'Stray Animal').count()

26760

How many service requests that are assigned to the Field Operations department (dept_division) are not classified as "Officer Standby" request type (service_request_type)?

In [8]:
(df
 .select('dept_division')
 .where(df.dept_division == 'Field Operations')
 .where(df.service_request_type != 'Officer Standby')
 .count())

113902

Create a new DataFrame without any information related to dates or location.

In [9]:
df_no_dates = df.drop('case_opened_date', 'case_closed_date', 'SLA_due_date', 'request_address', 'council_district')

In [10]:
df_no_dates.show(5)

+----------+---------+-------------------+-----------+----------------+--------------------+-----------+-----------+---------+
|   case_id|case_late|      num_days_late|case_closed|   dept_division|service_request_type|   SLA_days|case_status|source_id|
+----------+---------+-------------------+-----------+----------------+--------------------+-----------+-----------+---------+
|1014127332|       NO| -998.5087616000001|        YES|Field Operations|        Stray Animal|      999.0|     Closed| svcCRMLS|
|1014127333|       NO|-2.0126041669999997|        YES|     Storm Water|Removal Of Obstru...|4.322222222|     Closed| svcCRMSS|
|1014127334|       NO|       -3.022337963|        YES|     Storm Water|Removal Of Obstru...|4.320729167|     Closed| svcCRMSS|
|1014127335|       NO|       -15.01148148|        YES|Code Enforcement|Front Or Side Yar...|16.29188657|     Closed| svcCRMSS|
|1014127336|      YES|0.37216435200000003|        YES|Field Operations|Animal Cruelty(Cr...|      0.125|     Cl

Read dept.csv into a Spark DataFrame. Inspect the dept_name column. Replace the missing values with "other".

In [11]:
df = spark.read.csv('./dept.csv', header=True)

In [12]:
df.show()

+--------------------+--------------------+----------------------+-------------------+
|       dept_division|           dept_name|standardized_dept_name|dept_subject_to_SLA|
+--------------------+--------------------+----------------------+-------------------+
|     311 Call Center|    Customer Service|      Customer Service|                YES|
|               Brush|Solid Waste Manag...|           Solid Waste|                YES|
|     Clean and Green|Parks and Recreation|    Parks & Recreation|                YES|
|Clean and Green N...|Parks and Recreation|    Parks & Recreation|                YES|
|    Code Enforcement|Code Enforcement ...|  DSD/Code Enforcement|                YES|
|Code Enforcement ...|Code Enforcement ...|  DSD/Code Enforcement|                YES|
|Code Enforcement ...|                null|  DSD/Code Enforcement|                YES|
|   Dangerous Premise|Code Enforcement ...|  DSD/Code Enforcement|                YES|
|Dangerous Premise...|Code Enforcement ...|

In [13]:
df = df.na.fill('Other', ['dept_name']).show()

+--------------------+--------------------+----------------------+-------------------+
|       dept_division|           dept_name|standardized_dept_name|dept_subject_to_SLA|
+--------------------+--------------------+----------------------+-------------------+
|     311 Call Center|    Customer Service|      Customer Service|                YES|
|               Brush|Solid Waste Manag...|           Solid Waste|                YES|
|     Clean and Green|Parks and Recreation|    Parks & Recreation|                YES|
|Clean and Green N...|Parks and Recreation|    Parks & Recreation|                YES|
|    Code Enforcement|Code Enforcement ...|  DSD/Code Enforcement|                YES|
|Code Enforcement ...|Code Enforcement ...|  DSD/Code Enforcement|                YES|
|Code Enforcement ...|               Other|  DSD/Code Enforcement|                YES|
|   Dangerous Premise|Code Enforcement ...|  DSD/Code Enforcement|                YES|
|Dangerous Premise...|Code Enforcement ...|

## Part 2

    1. Convert the council_district column to a string column.
    2. Extract the year from the case_closed_date column.
    3. Convert num_days_late from days to hours in new columns num_hours_late.
    4. Convert the case_late column to a boolean column.
    5. Convert the SLA_days columns to a double column.
    6. Pull it all together

In [14]:
df = spark.read.csv('./case.csv', header=True)

Convert the council_district column to a string column.

In [15]:
df.council_district.cast('string')

Column<b'CAST(council_district AS STRING)'>

In [16]:
df.dtypes

[('case_id', 'string'),
 ('case_opened_date', 'string'),
 ('case_closed_date', 'string'),
 ('SLA_due_date', 'string'),
 ('case_late', 'string'),
 ('num_days_late', 'string'),
 ('case_closed', 'string'),
 ('dept_division', 'string'),
 ('service_request_type', 'string'),
 ('SLA_days', 'string'),
 ('case_status', 'string'),
 ('source_id', 'string'),
 ('request_address', 'string'),
 ('council_district', 'string')]

In [17]:
df.show()

+----------+----------------+----------------+------------+---------+-------------------+-----------+----------------+--------------------+------------------+-----------+---------+--------------------+----------------+
|   case_id|case_opened_date|case_closed_date|SLA_due_date|case_late|      num_days_late|case_closed|   dept_division|service_request_type|          SLA_days|case_status|source_id|     request_address|council_district|
+----------+----------------+----------------+------------+---------+-------------------+-----------+----------------+--------------------+------------------+-----------+---------+--------------------+----------------+
|1014127332|     1/1/18 0:42|    1/1/18 12:29|9/26/20 0:42|       NO| -998.5087616000001|        YES|Field Operations|        Stray Animal|             999.0|     Closed| svcCRMLS|2315  EL PASO ST,...|               5|
|1014127333|     1/1/18 0:46|     1/3/18 8:11| 1/5/18 8:30|       NO|-2.0126041669999997|        YES|     Storm Water|Remova

In [18]:
from pyspark.sql.functions import *
import pyspark.sql.functions as F

Extract the year from the case_closed_date column.

In [19]:
df_time = df.select(to_timestamp(df.case_closed_date, 'M/d/y H:mm').cast('string').alias('time'))

In [20]:
df_time.select(df_time.time, substring(df_time.time, 0,4)).show()

+-------------------+---------------------+
|               time|substring(time, 0, 4)|
+-------------------+---------------------+
|2018-01-01 12:29:00|                 2018|
|2018-01-03 08:11:00|                 2018|
|2018-01-02 07:57:00|                 2018|
|2018-01-02 08:13:00|                 2018|
|2018-01-01 13:29:00|                 2018|
|2018-01-01 14:38:00|                 2018|
|2018-01-02 15:32:00|                 2018|
|2018-01-02 15:32:00|                 2018|
|2018-01-02 15:32:00|                 2018|
|2018-01-02 15:32:00|                 2018|
|2018-01-02 15:32:00|                 2018|
|2018-01-02 15:32:00|                 2018|
|2018-01-02 15:33:00|                 2018|
|2018-01-02 15:32:00|                 2018|
|2018-01-02 15:33:00|                 2018|
|2018-01-02 15:33:00|                 2018|
|2018-01-02 15:33:00|                 2018|
|2018-01-02 15:33:00|                 2018|
|2018-01-02 15:33:00|                 2018|
|2018-01-02 15:33:00|           

Convert num_days_late from days to hours in new columns num_hours_late.

In [21]:
df.withColumn('num_hours_late', df.num_days_late*24).show()

+----------+----------------+----------------+------------+---------+-------------------+-----------+----------------+--------------------+------------------+-----------+---------+--------------------+----------------+-------------------+
|   case_id|case_opened_date|case_closed_date|SLA_due_date|case_late|      num_days_late|case_closed|   dept_division|service_request_type|          SLA_days|case_status|source_id|     request_address|council_district|     num_hours_late|
+----------+----------------+----------------+------------+---------+-------------------+-----------+----------------+--------------------+------------------+-----------+---------+--------------------+----------------+-------------------+
|1014127332|     1/1/18 0:42|    1/1/18 12:29|9/26/20 0:42|       NO| -998.5087616000001|        YES|Field Operations|        Stray Animal|             999.0|     Closed| svcCRMLS|2315  EL PASO ST,...|               5|     -23964.2102784|
|1014127333|     1/1/18 0:46|     1/3/18 8:1

Convert the case_late column to a boolean column.

In [22]:
df.select('case_id', 'case_late').withColumn('case_late', df.case_late.cast('boolean')).show()

+----------+---------+
|   case_id|case_late|
+----------+---------+
|1014127332|    false|
|1014127333|    false|
|1014127334|    false|
|1014127335|    false|
|1014127336|     true|
|1014127337|    false|
|1014127338|    false|
|1014127339|    false|
|1014127340|    false|
|1014127341|    false|
|1014127342|    false|
|1014127343|    false|
|1014127344|    false|
|1014127345|    false|
|1014127346|    false|
|1014127347|    false|
|1014127348|    false|
|1014127349|    false|
|1014127350|    false|
|1014127351|    false|
+----------+---------+
only showing top 20 rows



Convert the SLA_days columns to a double column.

In [23]:
df.select('SLA_days').show()

+------------------+
|          SLA_days|
+------------------+
|             999.0|
|       4.322222222|
|       4.320729167|
|       16.29188657|
|             0.125|
|       30.08446759|
|       16.06429398|
|16.063796300000003|
|       16.06333333|
|        16.0628588|
|       16.06237269|
|       16.06104167|
|       16.06059028|
|       16.06011574|
|       16.05953704|
|       16.05907407|
|       16.05864583|
|       16.05819444|
|       16.05775463|
|       16.05733796|
+------------------+
only showing top 20 rows



In [24]:
SLA_double = df.select('SLA_days').withColumn('SLA_days', df.SLA_days.cast('double'))

In [25]:
SLA_double.dtypes

[('SLA_days', 'double')]

Pull it all together

In [26]:
(df
 .withColumn('num_hours_late', df.num_days_late*24)
 .withColumn('case_late', df.case_late.cast('boolean'))
 .withColumn('SLA_days', df.SLA_days.cast('double'))
 .show())

+----------+----------------+----------------+------------+---------+-------------------+-----------+----------------+--------------------+------------------+-----------+---------+--------------------+----------------+-------------------+
|   case_id|case_opened_date|case_closed_date|SLA_due_date|case_late|      num_days_late|case_closed|   dept_division|service_request_type|          SLA_days|case_status|source_id|     request_address|council_district|     num_hours_late|
+----------+----------------+----------------+------------+---------+-------------------+-----------+----------------+--------------------+------------------+-----------+---------+--------------------+----------------+-------------------+
|1014127332|     1/1/18 0:42|    1/1/18 12:29|9/26/20 0:42|    false| -998.5087616000001|        YES|Field Operations|        Stray Animal|             999.0|     Closed| svcCRMLS|2315  EL PASO ST,...|               5|     -23964.2102784|
|1014127333|     1/1/18 0:46|     1/3/18 8:1

## Part 3

    1. Create a DataFrame with all combinations of council_district and service_request_type (regardless of whether the combination is observed in the data).
    2. Join the case data with the source and department data.
    3. Are there any cases that do not have a request source?

In [27]:
council = df.select('dept_division').distinct()

In [28]:
service = df.select('service_request_type').distinct()

In [29]:
council.crossJoin(service).show(10)

+-------------+--------------------+
|dept_division|service_request_type|
+-------------+--------------------+
|Miscellaneous|Minimum Housing-O...|
|Miscellaneous|        Tree Removal|
|Miscellaneous| Service Information|
|Miscellaneous|    Sign Maintenance|
|Miscellaneous|Park Building Mai...|
|  Solid Waste|Minimum Housing-O...|
|  Solid Waste|        Tree Removal|
|  Solid Waste| Service Information|
|  Solid Waste|    Sign Maintenance|
|  Solid Waste|Park Building Mai...|
+-------------+--------------------+
only showing top 10 rows



In [30]:
all_combos = council.crossJoin(service)
type(all_combos)

pyspark.sql.dataframe.DataFrame

Join the case data with the source and department data.

In [31]:
case = spark.read.csv('./case.csv', header=True)
source = spark.read.csv('./source.csv', header=True)
dept = spark.read.csv('./dept.csv', header=True)

In [32]:
source.show()
dept.show()
case.show()

+---------+--------------------+
|source_id|     source_username|
+---------+--------------------+
|   100137|    Merlene Blodgett|
|   103582|         Carmen Cura|
|   106463|     Richard Sanchez|
|   119403|      Betty De Hoyos|
|   119555|      Socorro Quiara|
|   119868| Michelle San Miguel|
|   120752|      Eva T. Kleiber|
|   124405|           Lori Lara|
|   132408|       Leonard Silva|
|   135723|        Amy Cardenas|
|   136202|    Michelle Urrutia|
|   136979|      Leticia Garcia|
|   137943|    Pamela K. Baccus|
|   138605|        Marisa Ozuna|
|   138650|      Kimberly Green|
|   138650|Kimberly Green-Woods|
|   138793| Guadalupe Rodriguez|
|   138810|       Tawona Martin|
|   139342|     Jessica Mendoza|
|   139344|        Isis Mendoza|
+---------+--------------------+
only showing top 20 rows

+--------------------+--------------------+----------------------+-------------------+
|       dept_division|           dept_name|standardized_dept_name|dept_subject_to_SLA|
+-------

In [33]:
df1 = case.join(dept, case.dept_division == dept.dept_division, "full_outer")

In [34]:
joined = df1.join(source, df1.source_id == source.source_id, "full_outer").toPandas()

In [35]:
joined

,case_id,case_opened_date,case_closed_date,SLA_due_date,case_late,num_days_late,case_closed,dept_division,service_request_type,SLA_days,case_status,source_id,request_address,council_district,dept_division,dept_name,standardized_dept_name,dept_subject_to_SLA,source_id,source_username
0,1013554415,5/9/17 13:55,5/10/17 10:58,5/11/17 13:55,NO,-1.12275463,YES,Miscellaneous,Dead Animal - Cat,2.0,Closed,136202,"1039 UTOPIA LANE, San Antonio, 78223",3,Miscellaneous,Solid Waste Management,Solid Waste,YES,136202,Michelle Urrutia
1,1014423136,4/13/18 15:43,4/30/18 13:51,4/18/18 15:43,YES,11.92177083,YES,Solid Waste,Solid Waste - Miscellaneous,5.0,Closed,136202,"506 WALDER TRAIL, San Antonio, 78260",9,Solid Waste,Solid Waste Management,Solid Waste,YES,136202,Michelle Urrutia
2,1014442689,4/20/18 8:48,4/30/18 14:03,4/25/18 8:48,YES,5.219166667,YES,Solid Waste,Solid Waste - Miscellaneous,5.0,Closed,136202,"7609 ORLAND PARK, San Antonio, 78213",1,Solid Waste,Solid Waste Management,Solid Waste,YES,136202,Michelle Urrutia
3,1014659092,7/4/18 8:43,7/4/18 9:27,7/9/18 8:43,NO,-4.968969907,YES,Solid Waste,Solid Waste - Miscellaneous,5.0,Closed,136202,"327 NORTHAVEN DR, San Antonio, 78229",7,Solid Waste,Solid Waste Management,Solid Waste,YES,136202,Michelle Urrutia
4,1014705020,7/20/18 9:03,7/20/18 10:49,7/25/18 9:03,NO,-4.926840278,YES,Solid Waste,Solid Waste - Miscellaneous,5.0,Closed,136202,"13310 THORNRIDGE LN, San Antonio, 78232",9,Solid Waste,Solid Waste Management,Solid Waste,YES,136202,Michelle Urrutia
5,1014751853,8/6/18 11:40,None,8/11/18 11:40,NO,-2.486168981,NO,Solid Waste,Solid Waste - Miscellaneous,5.0,Open,136202,"7140 OLDHAM DR, Bexar County, 78239",0,Solid Waste,Solid Waste Management,Solid Waste,YES,136202,Michelle Urrutia
6,1013365010,2/24/17 14:52,3/2/17 16:52,3/1/17 14:52,YES,1.082939815,YES,Solid Waste,Solid Waste - Miscellaneous,5.0,Closed,136202,"1314 THORAIN BLVD, San Antonio, 78201",1,Solid Waste,Solid Waste Management,Solid Waste,YES,136202,Michelle Urrutia
7,1013370216,2/27/17 13:40,4/5/17 16:09,3/4/17 13:40,YES,32.10305556,YES,Solid Waste,Solid Waste - Miscellaneous,5.0,Closed,136202,"16810 MAPLE GLADE, San Antonio, 78247",10,Solid Waste,Solid Waste Management,Solid Waste,YES,136202,Michelle Urrutia
8,1013388125,3/6/17 11:45,3/6/17 11:53,3/11/17 11:45,NO,-4.994594907,YES,Solid Waste,Solid Waste - Miscellaneous,5.0,Closed,136202,"13202 LARKFIELD DR, San Antonio, 78233",10,Solid Waste,Solid Waste Management,Solid Waste,YES,136202,Michelle Urrutia
9,1013388141,3/6/17 11:48,3/6/17 11:57,3/11/17 11:48,NO,-4.993321759,YES,Solid Waste,Solid Waste - Miscellaneous,5.0,Closed,136202,"14807 BLUEMIST PASS, San Antonio, 78247",10,Solid Waste,Solid Waste Management,Solid Waste,YES,136202,Michelle Urrutia


In [36]:
joined.count()

case_id                   855269
case_opened_date          855269
case_closed_date          836936
SLA_due_date              855235
case_late                 855269
num_days_late             855235
case_closed               855269
dept_division             855269
service_request_type      855269
SLA_days                  855235
case_status               855269
source_id                 855269
request_address           855269
council_district          855269
dept_division             855269
dept_name                 855071
standardized_dept_name    855269
dept_subject_to_SLA       855269
source_id                 855269
source_username           855269
dtype: int64

Are there any cases that do not have a request source?

In [37]:
joined.source_id.isna().sum()

source_id    0
source_id    0
dtype: int64

In [38]:
joined.source_username.isna().sum()

0

## Part 4

    1. Who are the top 10 service request types in terms of number of requests?
    2. Who are the top 10 service request types in terms of average days late?
    3. Does number of days late depend on department?
    4. How do number of days late depend on department division and request type?

In [39]:
joined = joined.loc[:,~joined.columns.duplicated()]

In [40]:
joined.source_username.value_counts().head(10)

CRM_Listener           195416
svcCRMSS               113585
Terri Barron            19840
Nadine Olmos            19832
Mary A Torres           19570
Isis Mendoza            19238
Guadalupe Rodriguez     18735
svcCRMLS                18445
Delilah Sanchez         18103
George Ponce            17556
Name: source_username, dtype: int64

Who are the top 10 service request types in terms of average days late?

In [58]:
joined.groupby('service_request_type').head()

,case_id,case_opened_date,case_closed_date,SLA_due_date,case_late,num_days_late,case_closed,dept_division,service_request_type,SLA_days,case_status,source_id,request_address,council_district,dept_name,standardized_dept_name,dept_subject_to_SLA,source_username
0,1013554415,5/9/17 13:55,5/10/17 10:58,5/11/17 13:55,NO,-1.12275463,YES,Miscellaneous,Dead Animal - Cat,2.0,Closed,136202,"1039 UTOPIA LANE, San Antonio, 78223",3,Solid Waste Management,Solid Waste,YES,Michelle Urrutia
1,1014423136,4/13/18 15:43,4/30/18 13:51,4/18/18 15:43,YES,11.92177083,YES,Solid Waste,Solid Waste - Miscellaneous,5.0,Closed,136202,"506 WALDER TRAIL, San Antonio, 78260",9,Solid Waste Management,Solid Waste,YES,Michelle Urrutia
2,1014442689,4/20/18 8:48,4/30/18 14:03,4/25/18 8:48,YES,5.219166667,YES,Solid Waste,Solid Waste - Miscellaneous,5.0,Closed,136202,"7609 ORLAND PARK, San Antonio, 78213",1,Solid Waste Management,Solid Waste,YES,Michelle Urrutia
3,1014659092,7/4/18 8:43,7/4/18 9:27,7/9/18 8:43,NO,-4.968969907,YES,Solid Waste,Solid Waste - Miscellaneous,5.0,Closed,136202,"327 NORTHAVEN DR, San Antonio, 78229",7,Solid Waste Management,Solid Waste,YES,Michelle Urrutia
4,1014705020,7/20/18 9:03,7/20/18 10:49,7/25/18 9:03,NO,-4.926840278,YES,Solid Waste,Solid Waste - Miscellaneous,5.0,Closed,136202,"13310 THORNRIDGE LN, San Antonio, 78232",9,Solid Waste Management,Solid Waste,YES,Michelle Urrutia
5,1014751853,8/6/18 11:40,None,8/11/18 11:40,NO,-2.486168981,NO,Solid Waste,Solid Waste - Miscellaneous,5.0,Open,136202,"7140 OLDHAM DR, Bexar County, 78239",0,Solid Waste Management,Solid Waste,YES,Michelle Urrutia
21,1014128056,1/2/18 8:21,1/10/18 8:39,1/5/18 8:30,YES,5.00681713,YES,Waste Collection,Solid Waste Fees and Charges,3.00619213,Closed,136202,"3214 STONEY FORK, San Antonio, 78247",10,Solid Waste Management,Solid Waste,YES,Michelle Urrutia
22,1014128366,1/2/18 9:36,1/3/18 13:28,1/9/18 9:36,NO,-5.838321758999999,YES,Waste Collection,Cart Exchange Request,7.0,Closed,136202,"11822 SONG, San Antonio, 78216",9,Solid Waste Management,Solid Waste,YES,Michelle Urrutia
23,1014128555,1/2/18 10:07,1/3/18 15:29,1/5/18 10:07,NO,-1.7764930559999998,YES,Waste Collection,No Pickup,3.0,Closed,136202,"13703 BLUFF ROCK, San Antonio, 78216",9,Solid Waste Management,Solid Waste,YES,Michelle Urrutia
24,1014128764,1/2/18 10:45,1/8/18 13:58,1/9/18 10:45,NO,-0.865798611,YES,Waste Collection,Cart Exchange Request,7.0,Closed,136202,"6270 RIDGEBROOK, San Antonio, 78250",6,Solid Waste Management,Solid Waste,YES,Michelle Urrutia


Does number of days late depend on department?

In [65]:
# joined.groupby('dept_name').agg(count(joined.num_days_late))

How do number of days late depend on department division and request type?

In [86]:
# joined.sort_values('dept_division').groupby(joined.service_request_type).agg(sum(joined.num_days_late))